#imported from workspace

In [0]:
%run "/Users/harshith.r@diggibyte.com/common_functions"

In [0]:
%run "/Users/harshith.r@diggibyte.com/Schema"

Read the csv to DF

In [0]:
file_path ="/FileStore/tables/sample_csv.csv"
df_csv=read_file("csv",file_path,schema=Schema_employee,header=True)
display(df_csv)


id,name,age,city
1,John,25,New York
2,Jane,30,Los Angeles
3,David,35,Chicago
4,Sarah,28,Miami
5,Michael,32,Seattle
6,Sarah,28,Miami
7,Emma,40,San Francisco
8,Brian,45,null
9,Emily,null,Null
10,Matthew,27,Houston


Write the file

In [0]:
path="/FileStore/Bronze_layer/parquet_Employee"
write_file(df_csv,"parquet",path)

Read Json to df

In [0]:
path_json="dbfs:/FileStore/tables/sample_json_1.json"
df_json =read_file("json",path_json,schema=schema_project,multiline=True)
df_json.printSchema()

root
 |-- department: string (nullable = true)
 |-- id: long (nullable = true)
 |-- projects: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- status: string (nullable = true)
 |-- salary: long (nullable = true)



explode json

In [0]:
df_json_flat=explode_json(df_json)

In [0]:
path="/FileStore/Bronze_layer/parquet_Projects"
write_file(df_json_flat,"parquet",path)

Remane the columns

In [0]:
df_csv_column_renamed=rename_all_columns_to_lowercase(df_csv)
df_json_column_renamed=rename_all_columns_to_lowercase(df_json_flat)

Drop duplicates and replace null

In [0]:
df_csv_dropDup_null_replaced=remove_duplicates_replace_null(df_csv_column_renamed)
df_json_dropDup_null_replaced=remove_duplicates_replace_null(df_json_column_renamed)

In [0]:
path="/FileStore/Sliver_layer/parquet_Employee"
write_file(df_csv_dropDup_null_replaced,"parquet",path)

In [0]:
path="/FileStore/Sliver_layer/parquet_Projects"
write_file(df_json_dropDup_null_replaced,"parquet",path)

Join Table

In [0]:
df_joined=join_table(df_csv_dropDup_null_replaced,df_json_dropDup_null_replaced,'id','id','outer')

In [0]:
path="/FileStore/Gold_layer/parquet_Employee_Projects"
write_file(df_joined,"parquet",path)

In [0]:
display(df_joined)

id,name,age,city,department,salary,status,project_name
1,John,25,New York,Sales,50000,In Progress,Project B
1,John,25,New York,Sales,50000,Completed,Project A
2,Jane,30,Los Angeles,Marketing,60000,In Progress,Project D
2,Jane,30,Los Angeles,Marketing,60000,Completed,Project C
3,David,35,Chicago,null,null,null,null
4,Sarah,28,Miami,HR,55000,Completed,Project E
5,Michael,32,Seattle,IT,65000,In Progress,Project F
5,Michael,32,Seattle,IT,65000,In Progress,Project G
6,Sarah,28,Miami,HR,55000,Completed,Project H
6,Sarah,28,Miami,HR,55000,In Progress,Project I


Deleate records

In [0]:
values=[31,40,7,15]
df_final=deleate_record(df_joined,'id',values)
df_final.display()

id,name,age,city,department,salary,status,project_name
58,David,35,Chicago,null,null,null,null
12,Chris,31,Chicago,Marketing,70000,Completed,Project S
12,Chris,31,Chicago,Marketing,70000,In Progress,Project T
21,John,26,Dallas,Sales,55000,In Progress,Project H1
21,John,26,Dallas,Sales,55000,In Progress,Project G1
26,David,35,Chicago,HR,50000,In Progress,Project O1
26,David,35,Chicago,HR,50000,Completed,Project N1
48,Sarah,28,Miami,Finance,80000,Completed,Project Y2
48,Sarah,28,Miami,Finance,80000,Completed,Project X2
54,Emily,27,Houston,null,null,null,null
